In [79]:
%matplotlib notebook
import numpy as np
import matplotlib
from astropy.table import Table, vstack, join
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import plotting_resources as res
import matplotlib.gridspec as gs
plt.rc('text', usetex=True)
plt.ioff()

rauto_cut = 22.9
frauto_cut = 10**((rauto_cut-23.9)/(-2.5))

# Define some functions

In [80]:
def calc_outlier_frac(res, thresh):
    good = 0.0
    N = float(len(res))
    for r in res:
        if abs(r) < thresh:
            good+=1.0
    return (N-good)/N

def safediv(x,y):
    if y == 0: return 0
    return x/y

def find_histogram_edges(hist, bins):
    xs, ys = [], []
    for i in range(len(hist)):
        xs.append(bins[i])
        xs.append(bins[i+1])
        ys.append(hist[i])
        ys.append(hist[i])
    return xs, ys

# Load the catalog

In [81]:
#slitcat = Table().read('../catalogs/personal_catalogs/slits_phot_zs_cutonslitdist_rcl.csv', format='csv')
slitcat = Table().read('../catalogs/personal_catalogs/slits_phot_zs_cutonslitdist.csv', format='csv')

ldpcat = slitcat[np.where((slitcat['Rauto'] < rauto_cut) & 
                          (slitcat['zSpec'] > 0) & 
                          (slitcat['slit_distance'] < 1.0))]

# Split the catalog on Q

In [82]:
ldpcat['z_resid'] = ldpcat['zLDP'] - ldpcat['zSpec']
Q4 = ldpcat[np.where(ldpcat['Q'] == 4)]
Q3 = ldpcat[np.where(ldpcat['Q'] == 3)]
Q2 = ldpcat[np.where(ldpcat['Q'] == 2)]

# Make the outlier histogram

In [83]:
def calc_frac(nbad, ntot):
    if ntot <= 6:
        frac = 0.0
        upper, lower = 0.0, 0.0
    else:
        #if True:
        nbad = float(nbad)
        ntot = float(ntot)
        
        #frac = nbad/(ntot)
        frac = safediv(nbad, ntot)
        ntotinv = safediv(1.0, ntot)
        upper = np.sqrt((res.upper_limit(nbad)-nbad)**2*(ntotinv)**2 +
                        (res.upper_limit(ntot)-ntot)**2*(frac*ntotinv)**2)
        lower = np.sqrt((nbad-res.lower_limit(nbad))**2*(ntotinv)**2 +
                        (ntot-res.lower_limit(ntot))**2*(frac*ntotinv)**2)  
        
    return frac, upper, lower

In [84]:
outliers = Table({'low': np.arange(19., 23.1, 1.0)[:-1], 
                  'high': np.arange(19., 23.1, 1.0)[1:], 
                  'Nout_Q4': np.zeros(4),
                  'Nout_Q3': np.zeros(4),  
                  'Nout_Q2': np.zeros(4),
                  'Ntot_Q4': np.zeros(4),
                  'Ntot_Q3': np.zeros(4),  
                  'Ntot_Q2': np.zeros(4),
                  'f4': np.zeros(4),
                  'f4_up': np.zeros(4),
                  'f4_lo': np.zeros(4),
                  'f3': np.zeros(4),
                  'f3_up': np.zeros(4),
                  'f3_lo': np.zeros(4),
                  'f2': np.zeros(4),
                  'f2_up': np.zeros(4),
                  'f2_lo': np.zeros(4)})

In [85]:
for i in ['4','3','2']:
    var = 'Q'+i
    for row in outliers:
        row['Ntot_'+var] = len(eval(var)[np.where((eval(var)['Rauto'] > row['low']) &
                                                 (eval(var)['Rauto'] < row['high']))])
        row['Nout_'+var] = len(eval(var)[np.where((eval(var)['Rauto'] > row['low']) & 
                                                  (eval(var)['Rauto'] < row['high']) & 
                                                  (abs(eval(var)['zLDP']-eval(var)['zSpec']) > 0.02))])
        f, u, l = calc_frac(row['Nout_'+var], row['Ntot_'+var])
        row['f'+i] = f
        row['f'+i+'_up'] = u
        row['f'+i+'_lo'] = l
        

In [86]:
xo4, yo4 = find_histogram_edges(outliers['f4'], np.arange(19., 23.1, 1.0))
xo3, yo3 = find_histogram_edges(outliers['f3'], np.arange(19., 23.1, 1.0))
xo2, yo2 = find_histogram_edges(outliers['f2'], np.arange(19., 23.1, 1.0))

# Cut off error bars at 0 and 1

In [87]:
for i in ['4', '3', '2']:
    for row in outliers:
        if row['f'+i] + row['f'+i+'_up'] > 1.0:
            row['f'+i+'_up'] = 1.0 - row['f'+i]
        if row['f'+i] - row['f'+i+'_lo'] < 0.0:
            row['f'+i+'_lo'] = row['f'+i]

# Build the plot

In [88]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 18}

matplotlib.rc('font', **font)

In [89]:
hist_args = {'histtype': 'step', 'linewidth':1}
errbar_args = {'lw':0.5, 'capsize': 3, 'elinewidth': 0.5, 'capthick': 0.5, 'ms':0}
legend_args = {'fontsize': 13, 'frameon': False}

In [90]:
f, ((a0), (a1), (a2)) = plt.subplots(3,1, gridspec_kw={'height_ratios':[1,2,1]}, sharex=True)

# Plot top
a0.hist(Q4['Rauto'], bins=np.arange(19., 23.1, 0.5), color='black', **hist_args)
a0.hist(Q3['Rauto'], bins=np.arange(19., 23.1, 0.5), color='blue', **hist_args)
a0.hist(Q2['Rauto'], bins=np.arange(19., 23.1, 0.5), color='red', **hist_args)

# Plot middle
a1.axhline(0.0, c='0.5', lw=0.8)
a1.scatter(Q4['Rauto'], Q4['z_resid'], c='black', s=8, label=r'$Q=4$')
a1.scatter(Q3['Rauto'], Q3['z_resid'], c='blue', s=8, marker='^', label=r'$Q=3$')
a1.scatter(Q2['Rauto'], Q2['z_resid'], c='red', s=8, marker='s', label=r'$Q=2$')
a1.axvline(22.9, color='k', ls='--', lw=0.5)

# Plot bottom
xcents = np.array([19.5, 20.5, 21.5, 22.5])
a2.plot(xo4, 100.*np.array(yo4), '-k', linewidth=1)
a2.plot(xo3, 100.*np.array(yo3), '-b', linewidth=1)
a2.plot(xo2, 100.*np.array(yo2), '-r', linewidth=1)
a2.errorbar(xcents, 100.*outliers['f4'], yerr=[100.*outliers['f4_lo'], 100.*outliers['f4_up']], fmt='.k', **errbar_args)
a2.errorbar(xcents-0.1, 100.*outliers['f3'], yerr=[100.*outliers['f3_lo'], 100.*outliers['f3_up']], fmt='.b', **errbar_args)
a2.errorbar(xcents+0.1, 100.*outliers['f2'], yerr=[100.*outliers['f2_lo'], 100.*outliers['f2_up']], fmt='.r', **errbar_args)






# Config top
a0.set_xlim([19, 23])
a0.set_ylim([0, 80])
a0.set_ylabel(r'$\rm N$')
a0.set_yticks([0, 30, 60])
a0.set_yticklabels([0, 30, 60], fontsize=14)

# Config middle
a1.set_xlim([19, 23])
a1.set_ylim([-0.8, 0.3])
a1.set_ylabel(r'$\Delta z$')
a1.set_yticks([-0.6, -0.4, -0.2, 0.0, 0.2])
a1.set_yticklabels([-0.6, -0.4, -0.2, 0.0, 0.2], fontsize=14)
a1.legend(**legend_args)


# Config bottom
xticks_arr = [19, 20, 21, 22, 23]
a2.set_xticks(xticks_arr)
a2.set_xticklabels(xticks_arr, fontsize=14)
a2.set_ylim([-5., 105])
a2.set_ylabel(r'Outlier \%')
a2.set_xlabel(r'$R_{\rm AUTO}$')
a2.set_yticks(np.array([0.0, 0.2, 0.4, 0.6, 0.8, 1.0])*100.)
a2.set_yticklabels([0, 20, 40, 60, 80, 100], fontsize=14)


plt.tight_layout()
f.subplots_adjust(hspace=0, wspace=0.1)

In [92]:
plt.savefig('plots_02members/zvRauto.eps', dpi=300)
#plt.show()